In [186]:
import requests
import json
import pandas as pd
import numpy as np
import musicbrainzngs as mbz
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
# return requests as xml
mbz.set_format('xml')

In [3]:
# set user agent
mbz.set_useragent(
    "ComputerFace-PureBeing",
    "0.1",
    "https://github.com/byronkking/ComputerFace-PureBeing/",
)

### get artist info

In [4]:
# find artist id for FlyLo
request = mbz.search_artists("Flying Lotus")
for artist in request['artist-list']:
    print(u"{id}: {name}".format(id=artist['id'], name=artist["name"]))

fc7376fe-1a6f-4414-b4a7-83f50ed59c92: Flying Lotus
b4681cdc-4002-4521-8458-ac812f1b6d28: Lotus
7f7a18ff-90b3-496e-a25c-4bbb34f51edf: Lotus
f15386df-5950-46ba-a899-ed74dfaf0a42: Lotus
467778bf-cee8-408a-b22a-fde6a99087ad: Lotus
88d3235a-6d73-4a17-a98d-93099e9bbbac: Lotus
2891a20f-812f-4f8b-96a8-0b0361cd1785: Lotus
b2358a61-dd8f-47f4-a6ac-744c413c3fb0: Lotus
28a6c75c-5d8f-483f-9c8c-829b89fb0050: Lotus
2e27c86e-fe90-434c-9d35-8fe2e07738fe: Lotus
b78b664d-e2ba-435f-a71a-180e73c9c767: Lotus
e3251e2b-0db2-47c4-b684-2efa6df0a00e: Lotus
59eaea88-3089-4c24-9ec9-5a7ae73936be: Lotus
cc5c45ad-b296-4e08-8ffd-9de7ad4ee298: Lotus
bc559041-304b-4ecf-833b-0cb2ad1a4ba6: Lotus
2cc924f5-d6d8-477d-be49-e77575d6b4df: Flying
1c5e627b-d16f-41c3-a031-48b956829343: Flying
58d0cb0e-a088-46e6-811b-c093cc77573d: Lotus Crown
3554dfec-821a-4d34-b707-0cac889ae81e: Dark Lotus
e60b0737-095c-4cff-8e35-f24063d7e212: Crimson Lotus
b8ca92e7-3d72-4583-835b-3fefe4b8dcc9: Purple Lotus
28604798-0160-4673-937b-1d800220d762: Lot

In [5]:
# test getting FlyLo info with artist id
artist_id = "fc7376fe-1a6f-4414-b4a7-83f50ed59c92"
try:
    result = mbz.get_artist_by_id(artist_id)
except Exception as error:
    print("Something went wrong with the request: %s" % error)
else:
    artist = result["artist"]
    print("name:\t\t%s" % artist["name"])
    print("sort name:\t%s" % artist["sort-name"])

name:		Flying Lotus
sort name:	Flying Lotus


### get artist info

In [6]:
# test getting FlyLo info with artist id
artist_id = "fc7376fe-1a6f-4414-b4a7-83f50ed59c92"
result = mbz.get_artist_by_id(artist_id,includes=['releases'])

In [9]:
temp = pd.DataFrame.from_dict(result)

### get album info

#### albums and eps

In [154]:
result = mbz.get_artist_by_id(artist_id,includes=["release-groups","tags"], release_type=["album", "ep"])

In [153]:
result

{'artist': {'area': {'id': '489ce91b-6658-3307-9877-795b68554c98',
   'iso-3166-1-code-list': ['US'],
   'name': 'United States',
   'sort-name': 'United States'},
  'begin-area': {'id': '1f40c6e1-47ba-4e35-996f-fe6ee5840e62',
   'name': 'Los Angeles',
   'sort-name': 'Los Angeles'},
  'country': 'US',
  'gender': 'Male',
  'id': 'fc7376fe-1a6f-4414-b4a7-83f50ed59c92',
  'isni-list': ['0000000081899265'],
  'life-span': {'begin': '1983-10-07'},
  'name': 'Flying Lotus',
  'release-group-count': 20,
  'release-group-list': [{'first-release-date': '2015-07-04',
    'id': '13bee711-e15c-4eba-a9ad-ebeeb8d4bc98',
    'primary-type': 'Album',
    'secondary-type-list': ['DJ-mix'],
    'title': 'Lovers Melt 4',
    'type': 'Album'},
   {'first-release-date': '2011-01-12',
    'id': '1c892e7b-7e0c-4c16-948d-001d6d7f1b89',
    'primary-type': 'EP',
    'title': 'Cosmogramma Alt Takes',
    'type': 'EP'},
   {'first-release-date': '2008-08',
    'id': '290f058a-344c-3f4f-b3ab-bd3419fe4ce4',
    

In [26]:
for release_group in result["artist"]["release-group-list"]:
    print("({date}) {title} ({type})".format(date=release_group["first-release-date"],
                                            title=release_group["title"],type=release_group['type']))

(2015-07-04) Lovers Melt 4 (Album)
(2011-01-12) Cosmogramma Alt Takes (EP)
(2008-08) Shhh! (EP)
(2008-11-24) L.A. EP 2 X 3 (EP)
(2009) Tectonic Plate 2.3 (EP)
(2009-05-02) LA CD (Compilation)
(2010) IIOIO (Compilation)
(2009-02) Whole Wide World / Lit Up / Keep It Moving (EP)
(2008-06-04) Los Angeles (Album)
(2010-09-11) Pattern+Grid World (EP)
() 2009-04-05: Plastic People, London, UK (Live)
(2006-10-03) 1983 (Album)
(2014-10-06) You’re Dead! (Album)
(2007-10-01) Reset (EP)
() Raw Cartoons (Compilation)
(2008-06-30) L.A. EP 1 X 3 (EP)
(2009-08-17) L.A. EP 3 X 3 (EP)
(2010-04-21) Cosmogramma (Album)
(2012-09-26) Until the Quiet Comes (Album)
() 2010-06-18: Sonar 2010, Barcelona, SP (Live)


#### only albums

In [89]:
result1 = mbz.get_artist_by_id(artist_id,includes=["release-groups","tags"], release_type=["album"])

In [135]:
# get genres of albums
for release_group in result1["artist"]["release-group-list"]:
    if release_group['type'] == 'Album' and release_group['title'] != 'Lovers Melt 4':
        print("({date}) {tag} {title} ({type})".format(date=release_group["first-release-date"],
                                                tag=release_group["tag-list"],title=release_group["title"],type=release_group['type']))

(2008-06-04) [{'count': '2', 'name': 'beatgen'}, {'count': '3', 'name': 'electronic'}, {'count': '1', 'name': 'hip-hop'}, {'count': '2', 'name': 'instrumental'}] Los Angeles (Album)
(2006-10-03) [{'count': '1', 'name': 'electronic'}, {'count': '2', 'name': 'hip-hop'}, {'count': '2', 'name': 'instrumental'}] 1983 (Album)
(2014-10-06) [{'count': '1', 'name': 'electronic'}] You’re Dead! (Album)
(2010-04-21) [{'count': '2', 'name': 'electronic'}, {'count': '1', 'name': 'hip-hop idm'}, {'count': '1', 'name': 'hip-hop industrial'}] Cosmogramma (Album)
(2012-09-26) [{'count': '4', 'name': 'electronic'}] Until the Quiet Comes (Album)


In [155]:
newList = []
for record in result["artist"]["release-group-list"]:
    currentJson = {}
    currentJson['title'] = record.get('title')
    currentJson['date'] = record.get('first-release-date')
    currentJson['tag'] = record.get('tag-list')
    currentJson['type'] = record.get('type')
    newList.append(currentJson)

In [233]:
df = pd.DataFrame(newList)

In [234]:
df['year'] = df['date'].str[:4]

In [246]:
newdf = pd.DataFrame(df.groupby('year').type.value_counts())
newdf.columns = ['count']

In [248]:
temp1 = newdf.reset_index(0)

In [250]:
temp1['type'] = temp1.index

In [253]:
temp1

,year,count,type
type,,,
Live,,2,Live
Compilation,,1,Compilation
Album,2006,1,Album
EP,2007,1,EP
EP,2008,3,EP
Album,2008,1,Album
EP,2009,3,EP
Compilation,2009,1,Compilation
Album,2010,1,Album


In [ ]:
temp1.plot(x='year',y='count',type='bar')